In [6]:
from inspect import signature

import numpy as np
from numpy import ndarray
#
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray

import shell.shell
import shell.utils

import cProfile
import skcuda.linalg

In [7]:
from Ottergrad.autograd import Tensor
from Ottergrad.autograd import Func


In [8]:


gpu_device = cuda.Device(0)
device_attributes_tuples = gpu_device.get_attributes().items()
for item, count in device_attributes_tuples:
    print(item, count)

ASYNC_ENGINE_COUNT 3
CAN_MAP_HOST_MEMORY 1
CLOCK_RATE 1200000
COMPUTE_CAPABILITY_MAJOR 7
COMPUTE_CAPABILITY_MINOR 5
COMPUTE_MODE DEFAULT
CONCURRENT_KERNELS 1
ECC_ENABLED 0
GLOBAL_L1_CACHE_SUPPORTED 1
GLOBAL_MEMORY_BUS_WIDTH 192
GPU_OVERLAP 1
INTEGRATED 0
KERNEL_EXEC_TIMEOUT 1
L2_CACHE_SIZE 3145728
LOCAL_L1_CACHE_SUPPORTED 1
MANAGED_MEMORY 1
MAXIMUM_SURFACE1D_LAYERED_LAYERS 2048
MAXIMUM_SURFACE1D_LAYERED_WIDTH 32768
MAXIMUM_SURFACE1D_WIDTH 32768
MAXIMUM_SURFACE2D_HEIGHT 65536
MAXIMUM_SURFACE2D_LAYERED_HEIGHT 32768
MAXIMUM_SURFACE2D_LAYERED_LAYERS 2048
MAXIMUM_SURFACE2D_LAYERED_WIDTH 32768
MAXIMUM_SURFACE2D_WIDTH 131072
MAXIMUM_SURFACE3D_DEPTH 16384
MAXIMUM_SURFACE3D_HEIGHT 16384
MAXIMUM_SURFACE3D_WIDTH 16384
MAXIMUM_SURFACECUBEMAP_LAYERED_LAYERS 2046
MAXIMUM_SURFACECUBEMAP_LAYERED_WIDTH 32768
MAXIMUM_SURFACECUBEMAP_WIDTH 32768
MAXIMUM_TEXTURE1D_LAYERED_LAYERS 2048
MAXIMUM_TEXTURE1D_LAYERED_WIDTH 32768
MAXIMUM_TEXTURE1D_LINEAR_WIDTH 268435456
MAXIMUM_TEXTURE1D_MIPMAPPED_WIDTH 32768
MAXIM

In [9]:
skcuda.linalg.init()
W = np.ones((5,5), dtype=np.float32)
x = np.ones((1,5), dtype=np.float32)
W_gpu = gpuarray.to_gpu(W)
x_gpu = gpuarray.to_gpu(x)
a = np.dot(x, W)
print(a)
a = skcuda.linalg.dot(x_gpu, W_gpu)
print(a)

[[5. 5. 5. 5. 5.]]
[[5. 5. 5. 5. 5.]]


In [10]:

W_cpu = Tensor(np.ones((5,5)), device="cpu")
x_cpu = Tensor(np.ones((1,5)), device="cpu")
func_cpu = x_cpu@W_cpu
func_cpu = Func(func_cpu)
func_cpu.forward()
func_cpu.backward()
# print(W.getgrad())

In [12]:

W = Tensor(np.ones((5,5)), device="gpu")
x = Tensor(np.ones((1,5)), device="gpu")
func = x@W
func = Func(func)
func.forward()
func.backward()
# print(W.getgrad())
W_cpu.getgrad() == W.getgrad().get()

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])